In [ ]:
import yaml
import glob
import numpy as np
import pickle
import pandas as pd
import sqlite3


In [ ]:
LABEL = 'taken'
FEATURES_TO_DISCRETIZE = ['to_user_distance'
                          ,'to_user_elevation'
                          ,'total_earning'
                         ]

DISCRETE_FEATURES = ['day_risk_group_1'
                     ,'day_risk_group_2'
                     ,'day_risk_group_3'
                     ,'day_risk_group_4'
                    ]

REG_FEATURES = ['store_not_taken_rate_2d'
                ,'store_not_taken_rate_1d'
                ,'store_not_taken_rate_12h'
                ,'store_not_taken_rate_3h'
                ,'store_not_taken_rate_1h'
                ,'to_user_distance'
                ,'total_earning'
                ,'to_user_elevation'
               ]

In [ ]:
df = pd.read_csv('../dataset/raw_dataset.csv')


In [ ]:
df['day_of_week'] = pd.to_datetime(df['created_at']).apply(lambda x: '{dayofweek}'.format(dayofweek=x.dayofweek)).astype(str)

day_risk_groups = {
    'day_risk_group_1' : ['5']
    ,'day_risk_group_2' : ['0','3','4']
    ,'day_risk_group_3' : ['1','2']
    ,'day_risk_group_4' : ['6']
}


In [ ]:
for risk_group in day_risk_groups:
    df[risk_group] = (df.day_of_week.isin(day_risk_groups[risk_group])).astype(int)


In [ ]:
conn = sqlite3.connect('raw_dataset.db')
df.to_sql('couriers_rejection',conn, if_exists='replace')


In [ ]:
with open('sql_queries/store_not_taken_rates.sql', 'r') as query_file:
    store_rates_query = query_file.read()
    df_store_rates = pd.read_sql_query(store_rates_query, conn).fillna(0)


In [ ]:
df = pd.merge(left=df,right=df_store_rates, left_on='order_id', right_on='order_id')


In [ ]:

#discrete features
discretized_dataset = df[DISCRETE_FEATURES + REG_FEATURES]

quantiles = [0, .25, .75, .9, .95, 1.]

#get quantiles for each column
for feature in FEATURES_TO_DISCRETIZE:
    discretized_dataset['discrete_'+feature] = pd.qcut(df[feature], q=quantiles, labels=range(1,len(quantiles)))

#one-hot encoding
discretized_dataset = pd.get_dummies(discretized_dataset)

#add labels
discretized_dataset[LABEL] = pd.to_numeric(df[LABEL])

#print result
discretized_dataset.head(50)

In [ ]:
discretized_dataset.to_pickle('../dataset/discretized_dataset')

In [ ]:
# Features correlation coefficent to label
fi = pd.DataFrame({'feature': list(discretized_dataset),
                   'correlation': discretized_dataset.corr()[LABEL].abs(),
                   'true correlation': discretized_dataset.corr()[LABEL]}).\
                    sort_values('correlation', ascending = False)

fi[fi.feature!=LABEL].head(20).plot.bar(x='feature', y ='true correlation')